Toy NB 

Input : variable latente z de dimension Dz

Output : 

moyenne mu_x de dimension Dx
  
covariance sigma_x (Dx, Dx) définie positive et non diagonale
  
Doit scaler avec batch, sequence_length :

(Dz) => (Dx) + (Dx,Dx)

(L,Dz) => (L,Dx) + (L,Dx,Dx)

(B,L,Dz) => (B,L,Dx) + (B,L,Dx,Dx)

In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.distributions.multivariate_normal import MultivariateNormal as MVN

In [2]:
Dz = 4
Dx = 16
B = 32
N = 50

In [3]:
# diagonale de la matrice de covariance : pas de souci.

diagonale = nn.Linear(Dz, Dx)

z = torch.randn(Dz)
x = diagonale(z)
print(f"z shape: {z.shape} => x shape: {x.shape}")

z = torch.randn(N,Dz)
x = diagonale(z)
print(f"z shape: {z.shape} => x shape: {x.shape}")

z = torch.randn(B,N,Dz)
x = diagonale(z)
print(f"z shape: {z.shape} => x shape: {x.shape}")


z shape: torch.Size([4]) => x shape: torch.Size([16])
z shape: torch.Size([50, 4]) => x shape: torch.Size([50, 16])
z shape: torch.Size([32, 50, 4]) => x shape: torch.Size([32, 50, 16])


In [4]:
# matric triangulaire inferieure de la matrice de covariance

class TriangularLower(nn.Module):
    def __init__(self, Dz=Dz, Dx=Dx):
        super(TriangularLower, self).__init__()
        self.full = nn.Sequential(
            nn.Linear(Dz, Dx * Dx),
            nn.Unflatten(-1, (Dx, Dx)),
            )

    def forward(self, x):
        return torch.tril(self.full(x))
    
triangulaire = TriangularLower()

In [5]:
z = torch.randn(Dz)
L = triangulaire(z)
print(f"z shape: {z.shape} => L shape: {L.shape}")

z = torch.randn(N,Dz)
L = triangulaire(z)
print(f"z shape: {z.shape} => L shape: {L.shape}")

z = torch.randn(B,N,Dz)
L = triangulaire(z)
print(f"z shape: {z.shape} => L shape: {L.shape}")

z shape: torch.Size([4]) => L shape: torch.Size([16, 16])
z shape: torch.Size([50, 4]) => L shape: torch.Size([50, 16, 16])
z shape: torch.Size([32, 50, 4]) => L shape: torch.Size([32, 50, 16, 16])


In [6]:
print(L[4, 3])
print(L.transpose(-1, -2)[4, 3])

tensor([[-0.7918,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.8348, -1.2289,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-1.0379, -1.9188,  2.2531,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0453,  1.0335,  1.2678,  0.0320,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.0656, -1.2755, -0.0632,  1.1836,  0.0953,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.6644,  0.3225, -0.1978,  1.4753, -1.9900, -0.5815,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.6872, -0.0

In [7]:
class CovarianceMatrix(nn.Module):
    def __init__(self, Dz=Dz, Dx=Dx):
        super(CovarianceMatrix, self).__init__()
        self.diagonale = nn.Linear(Dz, Dx)
        self.full = nn.Sequential(
            nn.Linear(Dz, Dx * Dx),
            nn.Unflatten(-1, (Dx, Dx)),
            )

    def forward(self, z):
        D = torch.diag_embed(torch.exp(self.diagonale(z)))  # Diagonal elements > 0
        T = torch.tril(self.full(z), diagonal=-1)  # Lower triangular matrix without diagonal
        L = D + T
        C = torch.einsum('...ij,...jk->...ik', L, L.transpose(-1, -2))
        return C
    
covariance_matrix = CovarianceMatrix()

In [8]:
z = torch.randn(Dz)
C = covariance_matrix(z)
print(f"z shape: {z.shape} => C shape: {C.shape}")

z = torch.randn(N,Dz)
C = covariance_matrix(z)
print(f"z shape: {z.shape} => C shape: {C.shape}")

z = torch.randn(B,N,Dz)
C = covariance_matrix(z)
print(f"z shape: {z.shape} => C shape: {C.shape}")

z shape: torch.Size([4]) => C shape: torch.Size([16, 16])
z shape: torch.Size([50, 4]) => C shape: torch.Size([50, 16, 16])
z shape: torch.Size([32, 50, 4]) => C shape: torch.Size([32, 50, 16, 16])


In [9]:
sample_C = C[4, 3]

print(sample_C)

tensor([[ 3.9435e+00,  7.7925e-01, -1.9096e+00,  1.6269e+00, -2.5272e-01,
         -1.6761e+00,  1.5084e+00, -1.3799e+00, -5.7419e-01, -4.3216e-02,
          1.6997e+00,  2.5782e+00, -2.3575e-01,  1.2093e-01,  8.5835e-02,
         -1.4866e+00],
        [ 7.7925e-01,  8.6263e+00, -2.6181e-01, -1.3465e+00,  8.4034e-01,
          1.5706e+00,  1.5629e+00, -3.5443e+00,  3.2278e+00,  2.9954e+00,
          3.1033e-01,  2.8698e+00, -8.4483e-01,  2.0412e+00,  5.1449e-01,
          6.2226e-01],
        [-1.9096e+00, -2.6181e-01,  9.2772e+00, -1.3081e+00,  1.2961e-02,
          1.3979e+00,  7.5842e-01,  2.6656e+00, -2.4583e-01,  8.6481e-01,
         -7.9920e-01, -1.7183e+00,  4.2322e-01,  3.0312e+00, -1.4723e+00,
          7.3420e-01],
        [ 1.6269e+00, -1.3465e+00, -1.3081e+00,  4.3061e+00, -1.0930e+00,
         -2.9179e+00,  1.4964e+00,  1.6030e+00, -3.5819e+00,  1.0260e-02,
          2.5197e+00,  5.6585e-01,  7.5451e-02,  1.6937e+00,  1.8191e+00,
         -3.9038e-02],
        [-2.5272e-01

In [10]:
L = torch.linalg.cholesky(sample_C, upper=False)
print(L)

print(L @ L.T)

tensor([[ 1.9858e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 3.9241e-01,  2.9107e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-9.6162e-01,  3.9694e-02,  2.8898e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 8.1925e-01, -5.7304e-01, -1.7218e-01,  1.8102e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-1.2726e-01

In [11]:
# Instantiation de MVN avec des matrices de covariance non diagonales

z = torch.randn(Dz)
mu = diagonale(z)
C = covariance_matrix(z)
print(f"z shape: {z.shape} => mu shape: {mu.shape}, covar shape: {C.shape}")
mvn = MVN(loc=mu, covariance_matrix=C)
print(f"mvn loc: {mvn.loc.shape}, covariance_matrix: {mvn.covariance_matrix.shape}")
print(f"mvn batch_shape: {mvn.batch_shape}, event_shape: {mvn.event_shape}")
print()

z = torch.randn(N,Dz)
mu = diagonale(z)
C = covariance_matrix(z)
print(f"z shape: {z.shape} => mu shape: {mu.shape}, covar shape: {C.shape}")
mvn = MVN(loc=mu, covariance_matrix=C)
print(f"mvn loc: {mvn.loc.shape}, covariance_matrix: {mvn.covariance_matrix.shape}")
print(f"mvn batch_shape: {mvn.batch_shape}, event_shape: {mvn.event_shape}")
print()

z = torch.randn(B,N,Dz)
mu = diagonale(z)
C = covariance_matrix(z)
print(f"z shape: {z.shape} => mu shape: {mu.shape}, covar shape: {C.shape}")
mvn = MVN(loc=mu, covariance_matrix=C)
print(f"mvn loc: {mvn.loc.shape}, covariance_matrix: {mvn.covariance_matrix.shape}")
print(f"mvn batch_shape: {mvn.batch_shape}, event_shape: {mvn.event_shape}")
print()

z shape: torch.Size([4]) => mu shape: torch.Size([16]), covar shape: torch.Size([16, 16])
mvn loc: torch.Size([16]), covariance_matrix: torch.Size([16, 16])
mvn batch_shape: torch.Size([]), event_shape: torch.Size([16])

z shape: torch.Size([50, 4]) => mu shape: torch.Size([50, 16]), covar shape: torch.Size([50, 16, 16])
mvn loc: torch.Size([50, 16]), covariance_matrix: torch.Size([50, 16, 16])
mvn batch_shape: torch.Size([50]), event_shape: torch.Size([16])

z shape: torch.Size([32, 50, 4]) => mu shape: torch.Size([32, 50, 16]), covar shape: torch.Size([32, 50, 16, 16])


ValueError: Expected parameter covariance_matrix (Tensor of shape (32, 50, 16, 16)) of distribution MultivariateNormal(loc: torch.Size([32, 50, 16]), covariance_matrix: torch.Size([32, 50, 16, 16])) to satisfy the constraint PositiveDefinite(), but found invalid values:
tensor([[[[ 6.1311e-01,  9.6867e-02, -1.8256e-01,  ...,  8.5216e-02,
           -3.7406e-01, -1.0978e-01],
          [ 9.6867e-02,  6.1187e+00, -3.0513e-01,  ...,  6.9263e-02,
           -2.3341e+00, -1.0623e-01],
          [-1.8256e-01, -3.0513e-01,  1.7707e+00,  ...,  6.4690e-01,
           -2.0326e-01, -1.7755e-01],
          ...,
          [ 8.5216e-02,  6.9263e-02,  6.4690e-01,  ...,  3.0457e+00,
           -3.2580e-01, -9.8104e-01],
          [-3.7406e-01, -2.3341e+00, -2.0326e-01,  ..., -3.2580e-01,
            3.1916e+00, -1.4539e-01],
          [-1.0978e-01, -1.0623e-01, -1.7755e-01,  ..., -9.8104e-01,
           -1.4539e-01,  3.0551e+00]],

         [[ 1.2580e+00, -9.7726e-01, -1.0637e+00,  ..., -1.0888e+00,
 

In [12]:
# Test for positive definiteness of covariance matrix by brute force sampling...

N_TESTS = int(1e+5)

for _ in tqdm(range(N_TESTS)):
    
    z = torch.randn(B,N,Dz)
    mu = diagonale(z)
    C = covariance_matrix(z)
    mvn = MVN(loc=mu, covariance_matrix=C)

  0%|          | 0/100000 [00:00<?, ?it/s]


ValueError: Expected parameter covariance_matrix (Tensor of shape (32, 50, 16, 16)) of distribution MultivariateNormal(loc: torch.Size([32, 50, 16]), covariance_matrix: torch.Size([32, 50, 16, 16])) to satisfy the constraint PositiveDefinite(), but found invalid values:
tensor([[[[ 5.8506e-01, -9.9067e-01,  3.1407e-02,  ..., -7.3362e-01,
            3.2397e-01, -1.1008e-01],
          [-9.9067e-01,  1.8618e+00, -2.1245e-02,  ...,  1.1000e+00,
           -4.3463e-01,  2.2256e-01],
          [ 3.1407e-02, -2.1245e-02,  1.0783e+00,  ..., -5.8787e-01,
           -9.8706e-01,  7.9053e-01],
          ...,
          [-7.3362e-01,  1.1000e+00, -5.8787e-01,  ...,  7.8268e+00,
            1.1289e+00,  2.2967e+00],
          [ 3.2397e-01, -4.3463e-01, -9.8706e-01,  ...,  1.1289e+00,
            7.6800e+00,  1.6737e+00],
          [-1.1008e-01,  2.2256e-01,  7.9053e-01,  ...,  2.2967e+00,
            1.6737e+00,  1.0396e+01]],

         [[ 1.6187e+00, -4.1551e-01,  3.2606e-01,  ..., -1.8111e-01,
 